In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

In [ ]:
data_dir = '../input/traffic-signs-classification/myData'

In [ ]:
labels=pd.read_csv('../input/traffic-signs-classification/labels.csv')
labels[0:5]

In [ ]:
classid=list(labels['ClassId'])
Name=list(labels['Name'])

normal_mapping=dict(zip(Name,classid)) 
reverse_mapping=dict(zip(classid,Name)) 

def mapper(value):
    return reverse_mapping[value]

In [ ]:
labelnum0=[]
for file in os.listdir(data_dir):
    labelnum0+=[int(file)]
labelnum=sorted(labelnum0)
print(labelnum)
print(len(labelnum))

In [ ]:
dataset=[]
datalabels=[]
count=0
for file in labelnum:
    path=os.path.join(data_dir,str(file))
    for im in tqdm(os.listdir(path)):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(40,40))
        image=img_to_array(image)
        image=image/255.0
        dataset+=[image]
        datalabels+=[count]
    count+=1

In [ ]:
data=np.array(dataset)
datalabels=np.array(datalabels)

In [ ]:
n=len(dataset)
print(n)
N=[]
for i in range(n):
    N+=[i]
random.shuffle(N)

In [ ]:
train=data[N[0:(n//10)*8]]
trainlabels=datalabels[N[0:(n//10)*8]]
test=data[N[(n//10)*8:]]
testlabels=datalabels[N[(n//10)*8:]]

In [ ]:
labels1=to_categorical(trainlabels)
labels=np.array(labels1)

In [ ]:
trainx,testx,trainy,testy=train_test_split(train,labels,test_size=0.2,random_state=44)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.2,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [ ]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(40,40,3),include_top=False,weights='imagenet',pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(43, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=32),validation_data=(testx,testy),epochs=20)

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
img0='../input/traffic-signs-classification/myData/35/00004_00029.jpg'
load_img(img0,target_size=(200,200))

In [ ]:
image=load_img(img0,target_size=(40,40))
image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [ ]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
pred_name=mapper(value)
labelnum=int(img0.split('/')[4])
name=mapper(labelnum)
print("Prediction is {}.".format(pred_name))
print("Correct Answer is {}.".format(name))

In [ ]:
pred2=model.predict(test)
print(test.shape)
print(pred2.shape)

PRED=[]
for item in pred2:
    value2=np.argmax(item)      
    PRED+=[value2]
    
ANS=testlabels

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)